# Atlantic coral and hardbottom habitat

This indicator was used in the Southeast Blueprint 2023.

Code written by Rua Mordecai and modified by Amy Keister

Last run by Amy Keister on 23 June 2023. It took 50 minutes run.

In [34]:
import os, string
import arcpy

In [35]:
import time
start = time.time()

In [36]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
#SourceWorkspace= 
OutWorkspace = r"D:\SE_Blueprint_2023\4_Indicators\AtlanticCoralAndHardbottomFull\Working"

In [37]:
# define final output name
Out= r"D:\SE_Blueprint_2023\4_Indicators\AtlanticCoralAndHardbottomFull\AtlanticCoralAndHardbottom.tif"

In [38]:
# define raster used for cell size, extent, and snapping
MarineRaster= r"D:\SE_Blueprint_2023\2_MarineExtents\Marine_Extent_v1.tif"

In [39]:
# define inputs
# modeled hardbottom probabililty
shallowHardbottomData = r"D:\SE_Blueprint_2023\4_Indicators\AtlanticCoralAndHardbottom\InputData\SE_Hardbottom_Modeling_Datasets\Hardbottom_Final_Logistic_Prediction.tif"
deepHardbottomData = r"D:\SE_Blueprint_2023\4_Indicators\AtlanticCoralAndHardbottom\InputData\SoutheastUS_PredictiveModeling_DeepSeaCorals_Hardbottom\Model_Predictions\Hardbottom_Median_Prediction.tif"
# shipwreck data
wrecks = r"D:\SE_Blueprint_2023\4_Indicators\AtlanticCoralAndHardbottom\InputData\ENC_Wrecks\ENC_Wrecks.shp"
# artificial reefs
artificialReefs = r"D:\SE_Blueprint_2023\4_Indicators\AtlanticCoralAndHardbottom\InputData\Artificial_Reefs\Artificial_Reefs.shp"
# NOAA coral observations
NOAACoralObs = r"D:\SE_Blueprint_2023\4_Indicators\AtlanticCoralAndHardbottom\InputData\deep_sea_corals_6cb7_510e_0a83_clean.csv"
# SAMBA hardbottom layer
Hardbottom = "F:\\GIS_DATA\\Marine\\SABMA\\SABMA_Final_02122015_SeafloorCoastal\\Seafloor\\Data\\Seafloor.gdb\\Hardbottom"
# Atlantic subregion boundaries to remove data from outside the Atlantic
AtlanticMarine= r"D:\SE_Blueprint_2023\2_MarineExtents\Marine_SA_v1.tif"
AtlanticMarineSHP= r"D:\SE_Blueprint_2023\2_MarineExtents\Marine_SA_v1.shp"
# landform data for the Blake Plateau
BLandform= r"F:\GIS_DATA\Marine\BlakePlateauLandform_DerekSowers\ForUSFWS-20230622T134752Z-002\ForUSFWS\Blake_Landforms.tif"
# Gulf subregion boundaries to remove data that overlaps the gulf 
#GOM= r"D:\SE_Blueprint_2023\2_MarineExtents\Marine_GoM_v1.tif"
# Base blueprint extent as another way to try to keep nearshore in the atlantic
base= r"F:\GIS_DATA\SECAS\SE_Blueprint_2022\Southeast_Blueprint_2022_Data_Download\SEBlueprint20221215\Inputs\BaseBlueprint\1_ExtentLayers\BaseBlueprintExtent2022.tif"
# NLCD is used to remove land
NLCD= r"F:\GIS_DATA\LanduseLandcover\NLCD\NLCD_landcover_2019_release_all_files_20210604\nlcd_2019_land_cover_l48_20210604\nlcd_2019_land_cover_l48_20210604.img"

In [40]:
# Sometimes arcpro is fussy about not overwriting things
arcpy.env.overwriteOutput = True

### Start analysis

In [41]:
# Set the output workspace
arcpy.env.workspace = OutWorkspace

In [42]:
# print workspace to make sure I'm in the right spot
print(arcpy.env.workspace)

D:\SE_Blueprint_2023\4_Indicators\AtlanticCoralAndHardbottomFull\Working


### start to make zeros that capture the inland extent of this indicator
There will be someoverlap with the gulf hardbottom and we have decided that is okay right now

In [10]:
# buffer atlantic marine shapefile by 100 km to try to retain nearshore hardbottom
# 100 km was chosen by a quick visual exploration, this number might need to be tweaked
with arcpy.EnvManager(outputCoordinateSystem=sr):
    arcpy.analysis.Buffer(AtlanticMarineSHP, "Marine_SA_v1_Buffer100km.shp", "100 Kilometers", "FULL", "ROUND", "NONE", None, "PLANAR")

In [11]:
# add field to convert to raster
arcpy.management.CalculateField("Marine_SA_v1_Buffer100km.shp", "raster", "0", "PYTHON3", '', "SHORT")

<Result 'D:\\SE_Blueprint_2023\\4_Indicators\\AtlanticCoralAndHardbottomFull\\Working\\Marine_SA_v1_Buffer100km.shp'>

In [12]:
# convert to 30m pixels 
#with arcpy.EnvManager(outputCoordinateSystem=sr, extent=MarineRaster, snapRaster=AtlanticMarine, cellSize=AtlanticMarine):
#        arcpy.conversion.FeatureToRaster(in_features="Marine_SA_v1_Buffer100km.shp", field="raster", out_raster="SAMzero.tif", cell_size=AtlanticMarine)

### Prep Hardbottom Probability data

In [13]:
# Reclassify shallow hardbottom into quantiles
out_raster = arcpy.sa.Reclassify(shallowHardbottomData, "Value", "0 0.023 1;0.023 0.083 2;0.083 0.206 3;0.206 0.349 4;0.349 0.847 5", "DATA"); out_raster.save("shallowBinned.tif")

In [14]:
# Reclassify deep hardbottom into quantiles
out_raster = arcpy.sa.Reclassify(deepHardbottomData, "Value", "0 0.128 1;0.128 0.320 2;0.320 0.640 3;0.640 0.949 4;0.949 1 5", "DATA"); out_raster.save("deepBinned.tif")

In [15]:
# Combine shallow and deep hardbottom, use newer deepHardbottom data where there's overlap, and snap and reproject for Blueprint
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=deepHardbottomData, snapRaster=MarineRaster, cellSize=MarineRaster):
    out_raster = arcpy.sa.Con(arcpy.sa.IsNull("shallowBinned.tif"),"deepBinned.tif",arcpy.sa.Con(arcpy.sa.IsNull("deepBinned.tif"),"shallowBinned.tif","deepBinned.tif")); out_raster.save("prob.tif")

### Make a buffer of Hardbottom Probability data and combine it with the Atlantic Marine Buffer

In [16]:
# reclass all probabivalues to 0
out_raster = arcpy.sa.Con("prob.tif", 1, "", "Value > 0"); out_raster.save("prob0.tif")

In [17]:
# need to buffer the probability data by 100 km also
arcpy.conversion.RasterToPolygon("prob0.tif", "prob0vector.shp", "NO_SIMPLIFY", "Value", "SINGLE_OUTER_PART", None)
arcpy.analysis.Buffer("prob0vector.shp", "prob0vectorBuf100km.shp", "100 Kilometers", "FULL", "ROUND", "NONE", None, "PLANAR")

<Result 'D:\\SE_Blueprint_2023\\4_Indicators\\AtlanticCoralAndHardbottomFull\\Working\\prob0vectorBuf100km.shp'>

In [18]:
# combine the SA marine buffer with the buffer of the probability data
arcpy.analysis.Union("Marine_SA_v1_Buffer100km.shp #;prob0vectorBuf100km.shp #", "All_Buffer100km.shp", "ALL", None, "NO_GAPS")

<Result 'D:\\SE_Blueprint_2023\\4_Indicators\\AtlanticCoralAndHardbottomFull\\Working\\All_Buffer100km.shp'>

In [19]:
# convert buffer area to a raster
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=MarineRaster, snapRaster=AtlanticMarine, cellSize=AtlanticMarine):
        arcpy.conversion.FeatureToRaster(in_features="All_Buffer100km.shp", field="raster", out_raster="AllZero.tif", cell_size=AtlanticMarine)

### Prep landform data for the Blake Plateau
In this raster
1=Flat
2=Peak
3=Ridge
6=Slope
9=Valley

Areas that are peaks, ridges, and slopes are indicaors that there are coral

In [20]:
# pull out peaks, ridges, and slopes from the landform data
with arcpy.EnvManager(outputCoordinateSystem=sr, extent="AllZero.tif", snapRaster=AtlanticMarine, cellSize=AtlanticMarine):
    out_raster = arcpy.sa.Con(BLandform, "6", "", where_clause="Value IN(2,3,6)"); out_raster.save("BLandform2.tif")

### Prep anthropogenic hardbottom

In [21]:
# combine shipwrecks, artificial reefs, and oil and gas platforms
with arcpy.EnvManager(outputCoordinateSystem=sr, extent="AllZero.tif", snapRaster=AtlanticMarine, cellSize=AtlanticMarine):
    arcpy.management.Merge([wrecks,artificialReefs],"anthroHardbottom.shp", "", "NO_SOURCE_INFO")

In [22]:
# Buffer anthropogenic known features to use as separate class
arcpy.analysis.Buffer("anthroHardbottom.shp", "anthroHardbottomBuf.shp", "150 Meters", "FULL", "ROUND", "NONE", None, "PLANAR")

<Result 'D:\\SE_Blueprint_2023\\4_Indicators\\AtlanticCoralAndHardbottomFull\\Working\\anthroHardbottomBuf.shp'>

In [23]:
# add field to convert to raster
arcpy.management.CalculateField("anthroHardbottomBuf.shp", "raster", "7", "PYTHON3", '', "SHORT")

<Result 'D:\\SE_Blueprint_2023\\4_Indicators\\AtlanticCoralAndHardbottomFull\\Working\\anthroHardbottomBuf.shp'>

In [24]:
# convert to 30m pixels 
with arcpy.EnvManager(outputCoordinateSystem=sr, extent="AllZero.tif", snapRaster=AtlanticMarine, cellSize=AtlanticMarine):
        arcpy.conversion.FeatureToRaster(in_features="anthroHardbottomBuf.shp", field="raster", out_raster="anthro.tif", cell_size=AtlanticMarine)

In [25]:
# reclassify so that noData is 0 
#out_raster = arcpy.sa.Reclassify("anthroHardBottomRaster.tif", "Value", "NODATA 0;100 6", "DATA"); out_raster.save("anthroHardBottomRasterZ.tif")

### Prep NOAA observed coral

In [26]:
# convert NOAA coral observation csv to shapefile
with arcpy.EnvManager(outputCoordinateSystem=sr, extent="AllZero.tif", snapRaster=AtlanticMarine, cellSize=AtlanticMarine):
    arcpy.management.XYTableToPoint(NOAACoralObs, "NOAAObs.shp", "longitude", "latitude", None, "GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision")

In [27]:
# select just priority hard corals from observation. These are based on previous Atlantic work with these data
out_raster = arcpy.management.SelectLayerByAttribute("NOAAObs.shp", "NEW_SELECTION", "Vernacular = 'stony coral (branching)' Or Vernacular = 'stony coral (cup coral)' Or Vernacular = 'stony coral (unspecified)' Or Vernacular = 'black coral' Or Vernacular = 'gorgonian coral'", None)

In [28]:
# Export the shapefile with the species removed
out_raster2 = arcpy.conversion.FeatureClassToFeatureClass(out_raster, OutWorkspace, "NOAAObsSelect.shp")

In [29]:
# Buffer corals.
arcpy.analysis.Buffer("NOAAObsSelect.shp", "NOAAObsSelectBuf.shp", "150 Meters", "FULL", "ROUND", "NONE", None, "PLANAR")

<Result 'D:\\SE_Blueprint_2023\\4_Indicators\\AtlanticCoralAndHardbottomFull\\Working\\NOAAObsSelectBuf.shp'>

In [30]:
# add field to convert to raster
arcpy.management.CalculateField("NOAAObsSelectBuf.shp", "raster", "8", "PYTHON3", '', "SHORT")

<Result 'D:\\SE_Blueprint_2023\\4_Indicators\\AtlanticCoralAndHardbottomFull\\Working\\NOAAObsSelectBuf.shp'>

In [31]:
# convert to 30m pixels 
with arcpy.EnvManager(outputCoordinateSystem=sr, extent="AllZero.tif", snapRaster=AtlanticMarine, cellSize=AtlanticMarine):
        arcpy.conversion.FeatureToRaster(in_features="NOAAObsSelectBuf.shp", field="raster", out_raster="NOAAObsR.tif", cell_size=AtlanticMarine)

### prep SABMA Observed data

In [32]:
# select, convert to raster, reclassify
with arcpy.EnvManager(outputCoordinateSystem=sr, extent="AllZero.tif", snapRaster=AtlanticMarine, cellSize=AtlanticMarine):
    arcpy.analysis.Select(Hardbottom, "sabmaObs.shp", "TEXT_DESC = '01. mapped hard bottom area'")
    arcpy.conversion.FeatureToRaster("sabmaObs.shp", "TEXT_DESC", "sabmaObsR.tif", cell_size=AtlanticMarine)
    out_raster = arcpy.sa.Reclassify("sabmaObsR.tif", "VALUE", "1 7", "DATA"); out_raster.save("sabmaObsR1.tif")

### combine data sources

In [33]:
# use cell statistics to combine all the data
out_raster = arcpy.sa.CellStatistics("prob.tif;anthro.tif;BLandform2.tif;NOAAObsR.tif;sabmaObsR1.tif;AllZero.tif", "MAXIMUM", "DATA", "SINGLE_BAND"); out_raster.save("ind.tif")

### use NLCD to remove land from indicator

In [34]:
# use NLCD to remove land
with arcpy.EnvManager(outputCoordinateSystem=sr, extent="AllZero.tif", snapRaster=AtlanticMarine, cellSize=AtlanticMarine):
    out_raster = arcpy.sa.Con(NLCD, "ind.tif", 0, "Value IN (0, 11)"); out_raster.save("indic.tif")  

### Finalize indicator



In [35]:
# export nonclipped version of indicator
#with arcpy.EnvManager(outputCoordinateSystem=sr, extent=MarineRaster, snapRaster=MarineRaster, cellSize=MarineRaster):
#    arcpy.management.CopyRaster("coralClasses.tif", IndicatorFileName, '', None, "255", "NONE", "NONE", "8_BIT_UNSIGNED", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

In [36]:
# use cell statistics to combine the Marine raster and base blueprint extent
# This will be used to remvoe deep marine areas outside the blueprint
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=MarineRaster, snapRaster=MarineRaster, cellSize=MarineRaster):
    out_raster = arcpy.sa.CellStatistics([MarineRaster, base], "MAXIMUM", "DATA", "SINGLE_BAND"); out_raster.save("FullExtent.tif")

In [37]:
# remove all areas outside the 100km buffer we made above
out_raster = arcpy.sa.ExtractByMask("indic.tif", "AllZero.tif"); out_raster.save("indica.tif")

In [43]:
# clip to full extent of base, atlantic, and gulf combined. Also make extent the same as the marine raster so it will work in zonation
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=MarineRaster, snapRaster=MarineRaster, cellSize=MarineRaster):
    out_raster = arcpy.sa.ExtractByMask("indica.tif", "FullExtent.tif"); out_raster.save("indicat.tif")

In [44]:
#export as 4 bit raster
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=MarineRaster, cellSize=MarineRaster):
    arcpy.management.CopyRaster("indicat.tif", Out, '', None, "15", "NONE", "NONE", "4_BIT", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

In [45]:
# set code block for next step
codeblock = """
def Reclass(value):  
    if value == 8:
        return '8 = Confirmed coral'
    elif value == 7:
        return '7 = Confirmed natural or human-created hardbottom (shipwrecks, artificial reefs)'
    elif value == 6:
        return '6 = Predicted cold water coral mounds (Blake Plateau)'
    elif value == 5:
        return '5 = Highest probability of hardbottom (>80th percentile)'
    elif value == 4:
        return '4 = High probability of hardbottom(>60th-80th percentile)'
    elif value == 3:
        return '3 = Medium probability of hardbottom (>40th-60th percentile)'
    elif value == 2:
        return '2 = Low probability of hardbottom (>20th-40th percentile)'
    elif value == 1:
        return '1 = Lowest probability of hardbottom (≤20th percentile)'    
    elif value == 0:
        return '0 = Not identified as hardbottom'      
"""

In [46]:
# add and calculate description field to hold indicator values
arcpy.management.CalculateField(Out, "descript", "Reclass(!value!)", "PYTHON3", codeblock, "TEXT")

<Result 'D:\\SE_Blueprint_2023\\4_Indicators\\AtlanticCoralAndHardbottomFull\\AtlanticCoralAndHardbottom.tif'>

In [47]:
# set code block for next step
codeblock = """
def Reclass1(Value):
	if Value == 8:
		return 0
	if Value == 7:
		return 23
	if Value == 6:
		return 58
	if Value == 5:
		return 132
	if Value == 4:
		return 195
	if Value == 3:
		return 244
	if Value == 2:
		return 255
	if Value == 1:
		return 255
	else:
		return 255
		
def Reclass2(Value):
	if Value == 8:
		return 96
	if Value == 7:
		return 118
	if Value == 6:
		return 133
	if Value == 5:
		return 138
	if Value == 4:
		return 140
	if Value == 3:
		return 156
	if Value == 2:
		return 178
	if Value == 1:
		return 202
	else:
		return 255
		
def Reclass3(Value):
	if Value == 8:
		return 166
	if Value == 7:
		return 187
	if Value == 6:
		return 196
	if Value == 5:
		return 191
	if Value == 4:
		return 167
	if Value == 3:
		return 145
	if Value == 2:
		return 163
	if Value == 1:
		return 191
	else:
		return 255
"""

In [48]:
# calculate Red field
arcpy.management.CalculateField(Out, "Red", "Reclass1(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Green field
arcpy.management.CalculateField(Out, "Green", "Reclass2(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Blue field
arcpy.management.CalculateField(Out, "Blue", "Reclass3(!Value!)", "PYTHON3", codeblock, "SHORT")

<Result 'D:\\SE_Blueprint_2023\\4_Indicators\\AtlanticCoralAndHardbottomFull\\AtlanticCoralAndHardbottom.tif'>

In [43]:
end = time.time()
print(end - start)

2937.105748653412
